
<div  style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://raw.githubusercontent.com/derar-alhussein/Databricks-Certified-Data-Engineer-Associate/main/Includes/images/bookstore_schema.png" alt="Databricks Learning" style="width: 600">
</div>

In [0]:
%python
files = dbutils.fs.ls("dbfs:/databricks-datasets/flowers/delta")
display(files)

In [0]:
CREATE TABLE orders AS
SELECT * FROM delta.`dbfs:/databricks-datasets/flowers/delta`

In [0]:
SELECT * FROM orders

## Overwriting Tables

In [0]:
CREATE OR REPLACE TABLE orders AS
SELECT * FROM delta.`dbfs:/databricks-datasets/flowers/delta`

In [0]:
DESCRIBE HISTORY orders

In [0]:
INSERT OVERWRITE orders
SELECT * FROM delta.`dbfs:/databricks-datasets/flowers/delta`

In [0]:
DESCRIBE HISTORY orders

In [0]:
ALTER TABLE orders ADD COLUMNS (ingest_time TIMESTAMP)

In [0]:
INSERT OVERWRITE orders
SELECT *, current_timestamp() FROM delta.`dbfs:/databricks-datasets/flowers/delta`

## Appending Data

In [0]:
INSERT INTO orders
SELECT *, current_timestamp() as ingest_time  FROM delta.`dbfs:/databricks-datasets/flowers/delta`

In [0]:
SELECT count(*) FROM orders

In [0]:
SELECT * FROM orders

## Merging Data

In [0]:
DROP TABLE IF EXISTS pais

In [0]:
%python
datos=[
  {"id":1,"pais":"Ecuador","capital":"Quito","poblacion":18.4},
  {"id":2,"pais":"Colombia","capital":"Bogota", "poblacion":50.3},
  {"id":3,"pais":"Peru","capital":"Lima", "poblacion":33.0}
]
df = spark.createDataFrame(datos)
df.write.mode("overwrite").saveAsTable("pais")

In [0]:
SELECT id,pais,capital,poblacion FROM pais

In [0]:
%python
datos=[
  {"id":4,"pais":"Bolivia","capital":"Sucre","poblacion":11},
]
df = spark.createDataFrame(datos)
df.createOrReplaceTempView("pais_update")

In [0]:
MERGE INTO pais t
USING pais_update s 
ON t.id=s.id 
WHEN MATCHED THEN UPDATE SET t.pais=s.pais,t.capital=s.capital,t.poblacion=s.poblacion
WHEN NOT MATCHED THEN INSERT *

In [0]:
SELECT * FROM pais